In [ ]:
import pandas as pd
import glob
import os

# Excelファイルが入っているフォルダ
folder_path = "lesson20260114\\order_new"

# フォルダ内の .xlsx ファイルをすべて取得
excel_files = glob.glob(os.path.join(folder_path, "*.xlsx"))

# 読み込んだデータを格納するリスト
df_list = []

for file in excel_files:
    df = pd.read_excel(file)
    # df["ファイル名"] = os.path.basename(file)  # どのファイルか分かるように
    df_list.append(df)

# すべて結合
all_df = pd.concat(df_list, ignore_index=True)

all_df.sum()

total_order = all_df.sum()


In [3]:
inventory_df = pd.read_excel("lesson20260114\\inventory.xlsx")

pickup_df = pd.read_excel("lesson20260114\\pickup.xlsx")

inventory_df

,日付,曜日,トマト,キャベツ,レタス,白菜,ほうれん草,大根,ニンジン
0,2023-05-01,Mon,106,70,101,64,95,65,68
1,2023-05-02,Tue,75,71,78,47,60,45,69
2,2023-05-03,Wed,61,81,82,53,51,25,89
3,2023-05-04,Thu,47,49,93,38,75,41,91
4,2023-05-05,Fri,67,52,67,21,97,28,56
5,2023-05-06,Sat,43,92,96,40,66,50,37
6,2023-05-07,Sun,81,39,74,18,104,38,77
7,2023-05-08,Mon,104,83,29,33,72,56,55
8,2023-05-09,Tue,31,36,84,35,92,44,78
9,2023-05-10,Wed,92,75,32,29,43,22,62


In [ ]:
df["日付"] = pd.to_datetime(df["日付"])

latest_date = df["日付"].max()
result = df[df["日付"] == latest_date]

print(result)


In [ ]:
inventory_df.iloc[-1, 2:]

トマト     91
キャベツ    73
Name: 13, dtype: object

In [11]:
pickup_df = pd.read_excel("lesson20260114\\pickup.xlsx")

pickup_df

,Unnamed: 0,トマト,キャベツ,レタス,白菜,ほうれん草,大根,ニンジン
0,しきい値,50,40,50,30,40,30,40
1,追加量,100,80,100,60,80,60,80
